# Exploring the Data
This is using Austin data. Looks like inside aribnb data is same format so it can be used for other cities


In [1]:
import io
import re
import numpy as np;
import pandas as pd;
import matplotlib;
import matplotlib.pyplot as plt;
import seaborn as sns;
import os

import warnings
warnings.filterwarnings('ignore')

In [2]:
#read data, specify index so it's easier to join and search using loc
folder = 'data/austin-airbnb/'
csv_path = os.path.join(folder, 'listings.csv')
listings = pd.read_csv(csv_path, index_col = 'id')

In [3]:
#shape
print("This is the shape of my DF: ", listings.shape)
print("This are the Data Types of my DF: ", listings.dtypes)
# (11520, 105) - Mulyono
# (11668, 105) - Daniela data from http://insideairbnb.com/get-the-data.html for March 17

This is the shape of my DF:  (11668, 105)
This are the Data Types of my DF:  listing_url                                      object
scrape_id                                         int64
last_scraped                                     object
name                                             object
summary                                          object
                                                 ...   
calculated_host_listings_count                    int64
calculated_host_listings_count_entire_homes       int64
calculated_host_listings_count_private_rooms      int64
calculated_host_listings_count_shared_rooms       int64
reviews_per_month                               float64
Length: 105, dtype: object


In [ ]:
# df1 = df.iloc[:,0:2]

# listings.loc[:, 'host_id':'host_is_superhost']

In [ ]:
# for col in listings.columns:
#     print(col)

In [4]:
#to get all columns, set option
pd.set_option('display.max_columns', 107)
#to get text with no truncation
pd.set_option('display.max_colwidth', -1)

In [21]:
df_1 = listings.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11668 entries, 2265 to 42931550
Columns: 105 entries, listing_url to reviews_per_month
dtypes: float64(23), int64(21), object(61)
memory usage: 9.4+ MB


AttributeError: 'NoneType' object has no attribute 'head'

In [5]:
#get all columns exported into a file
buffer = io.StringIO()
listings.info(buf=buffer, verbose=True, null_counts=True)
s = buffer.getvalue()

with open('listings_info.txt', 'w',encoding="utf-8") as f:
    f.write(s)

In [22]:
#clean up exported file from above so it can be imported to dataframe easily
with open('listings_info.txt', 'r') as f:
    lines = f.readlines()
with open('listings_info_clean.txt', 'w') as f:
    line_number = 0
    for line in lines:
        line_number = line_number + 1
        if line_number not in [1,2,3,4,5,109,110]:
            line1 = re.sub(' +', ' ', line)
            f.write(line1)

In [23]:
#read the output from cell above to data frame so it can be queried later
listing_columns = pd.read_csv("listings_info_clean.txt", delimiter=' ', header=None)

ParserError: Error tokenizing data. C error: Expected 6 fields in line 3, saw 7


In [14]:
listing_columns.head()

,0,1,2,3,4,5,6
0,NaN,#,Column,Non-Null,Count,Dtype,NaN
1,---,------,--------------,-----,NaN,NaN,NaN
2,NaN,0,listing_url,11668,non-null,object,NaN
3,NaN,1,scrape_id,11668,non-null,int64,NaN
4,NaN,2,last_scraped,11668,non-null,object,NaN


In [15]:
#filter columns with mostly blanks
filt = listing_columns[1]  < 7000
listing_columns[filt]

TypeError: '<' not supported between instances of 'str' and 'int'

In [11]:
#fcolumns with objecs and high counts
filt_object = listing_columns[3] == 'object'
filt_highcount = listing_columns[1] > 7000
listing_columns[filt_object][filt_highcount]

TypeError: '>' not supported between instances of 'str' and 'int'

In [ ]:
#looking at different text value for possible text analysis
filt = listings.access.notna()
listings.loc[filt, 'access'].head(2)

In [ ]:
#show all columns for one listing
listings.loc[702825]

In [ ]:
#availability for 702825?
calendar = pd.read_csv('data/austin-airbnb/calendar.csv')
filt = calendar.listing_id == 702825
calendar[filt].head()

In [ ]:
#review for 702825, comments doesn't indicate postive or negative
reviews = pd.read_csv('data/austin-airbnb/reviews.csv')
filt = reviews.listing_id == 702825
reviews[filt].head()

In [ ]:
#data shape
print('listings: ' + str(listings.shape))
print('calendar: ' + str(calendar.shape))
print('reviews: ' + str(reviews.shape))

In [ ]:
#new df with only int and float columns, exclude object type
listings_int_float = listings.select_dtypes(exclude=['object'], );
listings_int_float.head()

In [ ]:
#new df, select only few
listings_select = listings[[#'id','scrape_id','host_id','host_listings_count',
                          'host_total_listings_count','neighbourhood_cleansed',
                          #'latitude','longitude',
                          'accommodates','bathrooms',
                          'bedrooms','beds','guests_included',
                          'minimum_nights','maximum_nights',
                          #'minimum_minimum_nights','maximum_minimum_nights',
                          #'minimum_maximum_nights','maximum_maximum_nights',
                          #'minimum_nights_avg_ntm','maximum_nights_avg_ntm',
                          'availability_30','availability_60',
                          #'availability_90','availability_365',
                          'number_of_reviews','number_of_reviews_ltm'#,
                          #'review_scores_rating','review_scores_accuracy',
                          #'review_scores_cleanliness','review_scores_checkin',
                          #'review_scores_communication',
                          #'review_scores_location','review_scores_value','reviews_per_month',
                          #'calculated_host_listings_count',
                          #'calculated_host_listings_count_entire_homes',
                          #'calculated_host_listings_count_private_rooms',
                          #'calculated_host_listings_count_shared_rooms'
                         ]]



In [ ]:
# Using the Seaborn Pairplot to get cross plots between different attributes
sns.pairplot(listings_select);

In [ ]:
#heatmap for all int/float columns
plt.figure(figsize=(12,10))
correlations = listings_int_float.corr()
sns.heatmap(correlations,
           cmap='coolwarm');